# Leveraging Neo4j for Service Identification and Microservices Partitioning in Business Process Systems

# 1. Import Libraries

Import all necessary libraries for XML parsing, Neo4j interaction, GPU detection, and concurrent processing.

In [158]:
# Import Libraries
import os
import re
import xml.etree.ElementTree as ET
import html
import uuid
import pandas as pd
from neo4j import GraphDatabase
from pyvis.network import Network
import torch
from concurrent.futures import ThreadPoolExecutor

# 2. Check CUDA Availability

Detect whether CUDA (GPU) is available on your system. This information will be printed at the beginning of the notebook.

In [159]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

CUDA available: False
CUDA version: None
Using device: cpu


In [160]:
# Function to check CUDA availability
def check_cuda():
    if torch.cuda.is_available():
        print("CUDA is available. GPU will be used if applicable.")
        print(f"Device Name: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA is not available. Using CPU.")

# Execute Cuda
check_cuda()

CUDA is not available. Using CPU.


# 3. Define Connection to Neo4j

In [161]:
# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "170202Kcf"

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

In [162]:
def test_connection():
    try:
        with driver.session(database="erpbpmn") as session:
            result = session.run("RETURN 1 AS test")
            for record in result:
                print(f"Connection successful, test query result: {record['test']}")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

test_connection()

Connection successful, test query result: 1


# 4. Define Functions to Create Nodes and Relationships

In [163]:
def get_node_color(node_type, level):
    color_map = {
        'Task': {
            0: '#FFD700',  # Gold
            1: '#FFFACD',  # LemonChiffon
            2: '#FAFAD2',  # LightGoldenrodYellow
            3: '#FFFFE0'   # LightYellow
        },
        'StartEvent': {
            0: '#90EE90',  # LightGreen
            1: '#98FB98',  # PaleGreen
            2: '#8FBC8F',  # DarkSeaGreen
            3: '#3CB371'   # MediumSeaGreen
        },
        'EndEvent': {
            0: '#FF6347',  # Tomato
            1: '#FF4500',  # OrangeRed
            2: '#FF0000',  # Red
            3: '#DC143C'   # Crimson
        }
    }
    default_color = '#D3D3D3'
    return color_map.get(node_type, {}).get(level, default_color)

def create_node(tx, label, properties):
    # Check if node already exists
    check_query = (
        f"MATCH (n:{label} {{id: $properties.id}}) RETURN n"
    )
    existing = tx.run(check_query, properties=properties).single()
    if existing:
        return existing[0]
    
    color = get_node_color(label, properties.get('level', 0))
    query = (
        f"CREATE (n:{label} {{id: $properties.id}}) "
        "SET n += $properties, n.color = $color "
        "RETURN n"
    )
    result = tx.run(query, properties=properties, color=color)
    return result.single()[0]

def create_relationship_with_id(tx, source_id, target_id, rel_type, properties):
    # Check if relationship already exists
    check_query = """
    MATCH (a {id: $source_id})-[r]->(b {id: $target_id})
    WHERE type(r) = $rel_type
    RETURN r
    """
    existing = tx.run(check_query, 
                     source_id=source_id, 
                     target_id=target_id,
                     rel_type=rel_type).single()
    if existing:
        return existing[0]
    
    rel_color_map = {
        'SEQUENCE_FLOW': '#A9A9A9',
        'XOR_SPLIT': '#FF69B4',
        'XOR_JOIN': '#4169E1',
        'OR_SPLIT': '#FFD700',
        'OR_JOIN': '#00CED1'
    }
    color = rel_color_map.get(rel_type, '#696969')
    
    query = (
        f"MATCH (a {{id: $source_id}}), (b {{id: $target_id}}) "
        f"CREATE (a)-[r:{rel_type} {{id: $properties.id}}]->(b) "
        "SET r += $properties, r.color = $color "
        "RETURN r"
    )
    result = tx.run(query, source_id=source_id, target_id=target_id, properties=properties, color=color)
    record = result.single()
    if record:
        return record[0]
    else:
        print(f"Warning: Could not create relationship {rel_type} between {source_id} and {target_id}.")
        return None

def create_invisible_task(tx, source_id, target_id, gateway_type, level, module, activity=None):
    """Create invisible task node for gateway connections"""
    invisible_task_id = f"invisible_task_{str(uuid.uuid4())[:8]}"
    
    task_properties = {
        'id': invisible_task_id,
        'name': f"Invisible Task {gateway_type}",
        'level': level,
        'module': module,
        'activity': activity,
        'invisible': True
    }
    
    # Create invisible task
    task = create_node(tx, 'Task', task_properties)
    
    # Create relationships
    in_rel = create_relationship_with_id(
        tx,
        source_id,
        invisible_task_id,
        'SEQUENCE_FLOW',
        {'id': str(uuid.uuid4())}
    )
    
    out_rel = create_relationship_with_id(
        tx,
        invisible_task_id,
        target_id,
        'SEQUENCE_FLOW',
        {'id': str(uuid.uuid4())}
    )
    
    return task, in_rel, out_rel

# 5. Parse BPMN XML Files and Load into Neo4j

In [164]:
# Define functions to parse BPMN XML files
def clean_name(name):
    if not name:
        return ''
    name = re.sub('<[^<]+?>', '', str(name))
    name = html.unescape(name)
    return name.strip()

In [165]:
def parse_bpmn_xml(file_path, level, module, activity=None):
    tree = ET.parse(file_path)
    root = tree.getroot()

    elements = {
        'Task': [],
        'StartEvent': [],
        'EndEvent': []
    }
    flows = []
    gateways = {}

    def clean_id(raw_id, prefix):
        return f"{prefix}{raw_id}"

    # Set prefix for IDs
    id_prefix = f"{module}_{'_'.join(filter(None, [activity, str(level)]))}_"

    # Find root element containing process elements
    process_root = root.find('.//root') if root.find('.//root') is not None else root

    # Process all mxCell elements
    for cell in process_root.findall('.//mxCell'):
        cell_id = cell.get('id')
        if not cell_id or cell_id in ['0', '1']:  # Skip container cells
            continue

        style = cell.get('style', '')
        vertex = cell.get('vertex')
        edge = cell.get('edge')
        value = clean_name(cell.get('value', ''))

        if vertex == '1':  # Handle nodes
            clean_cell_id = clean_id(cell_id, id_prefix)
            
            if 'shape=mxgraph.bpmn.task' in style:
                elements['Task'].append({
                    'id': clean_cell_id,
                    'name': value if value else f"Task_{cell_id}",
                    'level': level,
                    'module': module,
                    'activity': activity
                })
            
            elif 'shape=mxgraph.bpmn.gateway2' in style:
                gateway_type = 'XOR'  # Default gateway type
                if 'gwType=parallel' in style:
                    gateway_type = 'AND'
                elif 'gwType=inclusive' in style:
                    gateway_type = 'OR'

                gateways[cell_id] = {
                    'id': clean_cell_id,
                    'type': gateway_type,
                    'level': level,
                    'module': module,
                    'activity': activity,
                    'incoming': [],
                    'outgoing': []
                }

            elif 'shape=mxgraph.bpmn.event' in style:
                event_type = 'StartEvent' if ('fillColor=#60a917' in style or 'symbol=general' in style) else 'EndEvent'
                elements[event_type].append({
                    'id': clean_cell_id,
                    'name': value if value else event_type,
                    'level': level,
                    'module': module,
                    'activity': activity
                })

        elif edge == '1':  # Handle edges/flows
            source = cell.get('source')
            target = cell.get('target')
            if source and target:
                flow = {
                    'id': clean_id(cell_id, id_prefix),
                    'sourceRef': clean_id(source, id_prefix),
                    'targetRef': clean_id(target, id_prefix),
                    'name': value if value else 'Sequence Flow',
                    'level': level,
                    'module': module,
                    'activity': activity
                }
                flows.append(flow)

                # Track gateway connections
                if source in gateways:
                    gateways[source]['outgoing'].append(flow)
                if target in gateways:
                    gateways[target]['incoming'].append(flow)

    return elements, flows, gateways

In [166]:
def process_gateways_and_flows(session, elements, flows, gateways):
    """
    Process gateways and create appropriate relationships in Neo4j with proper gateway handling
    """
    processed_flows = set()  # Track processed flows to avoid duplicates
    
    def create_flow(source_id, target_id, flow_type, properties):
        flow_key = f"{source_id}->{target_id}"
        if flow_key not in processed_flows:
            processed_flows.add(flow_key)
            session.execute_write(
                create_relationship_with_id,
                source_id,
                target_id,
                flow_type,
                properties
            )

    def handle_gateway(gateway, incoming_flows, outgoing_flows):
        gateway_type = gateway['type']  # XOR or OR
        gateway_id = gateway['id']
        
        if len(incoming_flows) == 1 and len(outgoing_flows) > 1:
            # Split pattern
            flow_type = f"{gateway_type}_SPLIT"
            source_id = incoming_flows[0]['sourceRef']
            
            # Create invisible task
            invisible_task_id = f"invisible_task_{gateway_id}"
            task_props = {
                'id': invisible_task_id,
                'name': f"Invisible {flow_type}",
                'level': gateway['level'],
                'module': gateway['module'],
                'activity': gateway['activity'],
                'invisible': True
            }
            
            session.execute_write(create_node, 'Task', task_props)
            
            # Connect incoming -> invisible -> outgoing
            create_flow(source_id, invisible_task_id, 'SEQUENCE_FLOW',
                       {'id': str(uuid.uuid4())})
            
            for out_flow in outgoing_flows:
                create_flow(invisible_task_id, out_flow['targetRef'],
                          flow_type, {'id': str(uuid.uuid4())})
                
        elif len(incoming_flows) > 1 and len(outgoing_flows) == 1:
            # Join pattern
            flow_type = f"{gateway_type}_JOIN"
            target_id = outgoing_flows[0]['targetRef']
            
            # Create invisible task
            invisible_task_id = f"invisible_task_{gateway_id}"
            task_props = {
                'id': invisible_task_id,
                'name': f"Invisible {flow_type}",
                'level': gateway['level'],
                'module': gateway['module'],
                'activity': gateway['activity'],
                'invisible': True
            }
            
            session.execute_write(create_node, 'Task', task_props)
            
            # Connect incoming -> invisible -> target
            for in_flow in incoming_flows:
                create_flow(in_flow['sourceRef'], invisible_task_id,
                          flow_type, {'id': str(uuid.uuid4())})
                
            create_flow(invisible_task_id, target_id, 'SEQUENCE_FLOW',
                       {'id': str(uuid.uuid4())})

    # Process regular sequence flows first
    non_gateway_flows = [
        flow for flow in flows
        if not any(gw['id'] in (flow['sourceRef'], flow['targetRef'])
                  for gw in gateways.values())
    ]

    for flow in non_gateway_flows:
        create_flow(flow['sourceRef'], flow['targetRef'], 'SEQUENCE_FLOW', flow)

    # Process gateways
    for gw_id, gateway in gateways.items():
        incoming = [f for f in flows if f['targetRef'] == gateway['id']]
        outgoing = [f for f in flows if f['sourceRef'] == gateway['id']]
        handle_gateway(gateway, incoming, outgoing)

    # Connect any disconnected nodes with invisible tasks
    def connect_disconnected_nodes():
        query = """
        MATCH (n) 
        WHERE NOT EXISTS((n)--())
        RETURN n.id AS id, labels(n)[0] as type,
               n.level as level, n.module as module
        """
        disconnected = session.run(query).data()
        
        for node in disconnected:
            # Find suitable connection
            connect_query = """
            MATCH (other)
            WHERE other.level = $level AND other.module = $module 
            AND NOT other.id = $id
            AND (other:Task OR 
                (other:StartEvent AND $type <> 'StartEvent') OR
                (other:EndEvent AND $type <> 'EndEvent'))
            RETURN other.id as target_id
            LIMIT 1
            """
            
            target = session.run(connect_query, 
                               level=node['level'],
                               module=node['module'],
                               id=node['id'],
                               type=node['type']).single()
            
            if target:
                session.execute_write(
                    create_invisible_task,
                    node['id'],
                    target['target_id'],
                    'SEQUENCE_FLOW',
                    node['level'],
                    node['module']
                )

    connect_disconnected_nodes()

In [167]:
def clear_database(session):
    """Clear database and set up constraints"""
    session.run("MATCH (n) DETACH DELETE n")
    
    # Create constraints for node uniqueness
    constraints = [
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n:StartEvent) REQUIRE n.id IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n:EndEvent) REQUIRE n.id IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Task) REQUIRE n.id IS UNIQUE"
    ]
    
    for constraint in constraints:
        try:
            session.run(constraint)
        except Exception as e:
            print(f"Warning: Constraint creation failed - {e}")

def verify_process_integrity(session):
    """Verify the integrity of the converted BPMN process"""
    integrity_checks = [
        # Check for disconnected nodes
        """
        MATCH (n)
        WHERE NOT EXISTS((n)--())
        RETURN n.id AS node_id, labels(n) AS type, n.level AS level, n.module AS module
        """,
        
        # Check for duplicate start events
        """
        MATCH (s:StartEvent)
        WITH s.level AS level, s.module AS module, collect(s) AS starts
        WHERE size(starts) > 1
        RETURN level, module, [s in starts | s.id] AS duplicate_starts
        """,
        
        # Check for duplicate end events
        """
        MATCH (e:EndEvent)
        WITH e.level AS level, e.module AS module, collect(e) AS ends
        WHERE size(ends) > 1
        RETURN level, module, [e in ends | e.id] AS duplicate_ends
        """,
        
        # Check for improper gateway connections
        """
        MATCH (n)-[r:XOR_SPLIT]->(m)
        WITH n, count(r) AS split_count
        WHERE split_count < 2
        RETURN n.id AS invalid_split_gateway, split_count
        """,
        
        """
        MATCH (n)-[r:XOR_JOIN]->(m)
        WITH m, count(r) AS join_count
        WHERE join_count < 2
        RETURN m.id AS invalid_join_gateway, join_count
        """
    ]
    
    results = {}
    for check in integrity_checks:
        result = session.run(check)
        records = result.data()
        if records:
            results[check] = records
    
    return results

def process_bpmn_file(session, filename, file_path, level, module, activity):
    print(f"\nProcessing {filename} at Level {level}...")
    
    # Parse and process BPMN file
    elements, flows, gateways = parse_bpmn_xml(file_path, level, module, activity)
    print(f"Found {sum(len(v) for v in elements.values())} elements, {len(flows)} flows, and {len(gateways)} gateways")
    
    # Create nodes
    for element_type, element_list in elements.items():
        for element in element_list:
            try:
                session.execute_write(create_node, element_type, element)
                print(f"Created {element_type}: {element['id']}")
            except Exception as e:
                print(f"Error creating {element_type}: {element['id']} - {e}")
    
    # Process gateways and create relationships
    try:
        process_gateways_and_flows(session, elements, flows, gateways)
    except Exception as e:
        print(f"Error processing flows and gateways: {e}")
    
    # Clean duplicate relationships
    try:
        session.run("""
        MATCH (a)-[r]->(b)
        WITH a, b, type(r) as type, collect(r) as rels
        WHERE size(rels) > 1
        WITH a, b, type, rels[0] as kept, rels[1..] as duplicates
        UNWIND duplicates as dupe
        DELETE dupe
        """)
    except Exception as e:
        print(f"Error cleaning duplicates: {e}")
    
    # Verify process integrity
    issues = verify_process_integrity(session)
    if issues:
        print("\nProcess integrity issues found:")
        for check, records in issues.items():
            print(f"\nCheck results:")
            for record in records:
                print(record)

# 6. Main Execution

In [168]:
def validate_bpmn_structure(session):
    """Validate the complete BPMN structure after import"""
    validation_queries = [
        # Check for disconnected nodes
        """
        MATCH (n)
        WHERE NOT EXISTS((n)--())
        RETURN n.id AS disconnected_node, labels(n) AS node_type
        """,
        
        # Validate gateway patterns
        """
        MATCH p=(n)-[r:XOR_SPLIT|OR_SPLIT]->(m)
        WITH n, count(r) as outgoing
        WHERE outgoing < 2
        RETURN n.id as node_id, 'Invalid split pattern' as issue, outgoing
        """,
        
        """
        MATCH p=(n)-[r:XOR_JOIN|OR_JOIN]->(m)
        WITH m, count(r) as incoming
        WHERE incoming < 2
        RETURN m.id as node_id, 'Invalid join pattern' as issue, incoming
        """,
        
        # Check for single start/end
        """
        MATCH (s:StartEvent)
        WITH s.level as level, s.module as module, count(s) as start_count
        WHERE start_count > 1
        RETURN level, module, start_count, 'Multiple start events' as issue
        """,
        
        """
        MATCH (e:EndEvent)
        WITH e.level as level, e.module as module, count(e) as end_count
        WHERE end_count > 1
        RETURN level, module, end_count, 'Multiple end events' as issue
        """
    ]
    
    validation_results = {}
    for query in validation_queries:
        result = session.run(query)
        records = result.data()
        if records:
            validation_results[query] = records
            
    return validation_results

In [169]:
def main():
    with driver.session(database="erpbpmn") as session:
        # Clear database and set up constraints
        clear_database(session)
        
        # Process BPMN files
        bpmn_dir = './assets'
        filenames = sorted([f for f in os.listdir(bpmn_dir) if f.endswith('.xml')])
        
        for filename in filenames:
            file_path = os.path.join(bpmn_dir, filename)
            
            # Parse filename for metadata
            if filename == "BPMN Level 0.xml":
                level = 0
                module = "ERP"
                activity = None
            else:
                match = re.match(r'BPMN\s+(.+?)\s+Level\s+(\d+)(?:\s+-\s+(.+))?\.xml', filename)
                if match:
                    module = match.group(1).strip()
                    level = int(match.group(2))
                    activity = match.group(3).strip() if match.group(3) else None
                else:
                    print(f"Skipping file {filename} - doesn't match naming pattern")
                    continue
            
            try:
                process_bpmn_file(session, filename, file_path, level, module, activity)
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        
        # Validate final structure
        print("\nValidating final BPMN structure...")
        validation_results = validate_bpmn_structure(session)
        if validation_results:
            print("\nValidation issues found:")
            for query, issues in validation_results.items():
                print("\nIssue type:", query)
                for issue in issues:
                    print(issue)
        else:
            print("All validations passed successfully!")

if __name__ == "__main__":
    main()


Processing BPMN Account Payable Level 1.xml at Level 1...
Found 3 elements, 2 flows, and 0 gateways
Created Task: Account Payable_1_5
Created StartEvent: Account Payable_1_3
Created EndEvent: Account Payable_1_7

Processing BPMN Account Payable Level 2.xml at Level 2...
Found 23 elements, 39 flows, and 11 gateways
Created Task: Account Payable_2_24
Created Task: Account Payable_2_25
Created Task: Account Payable_2_26
Created Task: Account Payable_2_28
Created Task: Account Payable_2_29
Created Task: Account Payable_2_30
Created Task: Account Payable_2_31
Created Task: Account Payable_2_32
Created Task: Account Payable_2_33
Created Task: Account Payable_2_43
Created Task: Account Payable_2_46
Created Task: Account Payable_2_48
Created Task: Account Payable_2_50
Created Task: Account Payable_2_52
Created Task: Account Payable_2_57
Created Task: Account Payable_2_64
Created Task: Account Payable_2_65
Created Task: Account Payable_2_68
Created Task: Account Payable_2_72
Created Task: Acco

In [170]:
def verify_data_import():
    with driver.session(database="erpbpmn") as session:
        result = session.run("MATCH (n) RETURN labels(n) AS Label, count(n) AS Count ORDER BY Count DESC")
        print("Node counts by label:")
        for record in result:
            print(f"{record['Label']}: {record['Count']}")

        result = session.run("MATCH ()-[r]->() RETURN type(r) AS RelationType, count(r) AS Count ORDER BY Count DESC")
        print("\nRelationship counts by type:")
        for record in result:
            print(f"{record['RelationType']}: {record['Count']}")

        result = session.run("MATCH (n) RETURN n.level AS Level, count(n) AS Count ORDER BY Level")
        print("\nNode counts by level:")
        for record in result:
            print(f"Level {record['Level']}: {record['Count']} nodes")

# 7. Execute Main Function

In [171]:
def run():
    main()
    verify_data_import()

if __name__ == "__main__":
    run()


Processing BPMN Account Payable Level 1.xml at Level 1...
Found 3 elements, 2 flows, and 0 gateways
Created Task: Account Payable_1_5
Created StartEvent: Account Payable_1_3
Created EndEvent: Account Payable_1_7

Processing BPMN Account Payable Level 2.xml at Level 2...
Found 23 elements, 39 flows, and 11 gateways
Created Task: Account Payable_2_24
Created Task: Account Payable_2_25
Created Task: Account Payable_2_26
Created Task: Account Payable_2_28
Created Task: Account Payable_2_29
Created Task: Account Payable_2_30
Created Task: Account Payable_2_31
Created Task: Account Payable_2_32
Created Task: Account Payable_2_33
Created Task: Account Payable_2_43
Created Task: Account Payable_2_46
Created Task: Account Payable_2_48
Created Task: Account Payable_2_50
Created Task: Account Payable_2_52
Created Task: Account Payable_2_57
Created Task: Account Payable_2_64
Created Task: Account Payable_2_65
Created Task: Account Payable_2_68
Created Task: Account Payable_2_72
Created Task: Acco

# 8. Visualize the Graph

In [172]:
def get_neo4j_data():
    with driver.session(database="erpbpmn") as session:
        nodes_query = """
        MATCH (n) 
        RETURN n.id AS id, labels(n) AS labels, n.name AS name, n.color AS color, n.level AS level, n.module AS module, n.activity AS activity
        """
        nodes = session.run(nodes_query).data()
        nodes_df = pd.DataFrame(nodes)

        relationships_query = """
        MATCH (a)-[r]->(b) 
        RETURN r.id AS id, type(r) AS type, a.id AS source, b.id AS target, r.color AS color, r.level AS level, r.module AS module, r.activity AS activity
        """
        relationships = session.run(relationships_query).data()
        relationships_df = pd.DataFrame(relationships)

    return nodes_df, relationships_df

def visualize_neo4j_graph(nodes_df, relationships_df):
    net = Network(height='750px', width='100%', directed=True, notebook=True, cdn_resources='remote')

    for _, row in nodes_df.iterrows():
        label = row['name'] if pd.notnull(row['name']) else row['id']

        net.add_node(
            row['id'],
            label=label,
            title=f"ID: {row['id']}<br>Type: {row['labels']}<br>Name: {row['name']}<br>Level: {row['level']}<br>Module: {row['module']}<br>Activity: {row['activity']}",
            color=row['color'],
            level=row['level']
        )

    for _, row in relationships_df.iterrows():
        net.add_edge(
            row['source'],
            row['target'],
            title=f"ID: {row['id']}<br>Type: {row['type']}<br>Level: {row['level']}<br>Module: {row['module']}<br>Activity: {row['activity']}",
            color=row['color'],
            arrows='to'
        )

    net.force_atlas_2based()

    net.show('neo4j_graph.html')

    from IPython.display import IFrame
    return IFrame('neo4j_graph.html', width='100%', height='750px')

In [173]:
nodes_df, relationships_df = get_neo4j_data()

print("Nodes:")
display(nodes_df.head())

print("\nRelationships:")
display(relationships_df.head())

graph_display = visualize_neo4j_graph(nodes_df, relationships_df)
graph_display

Nodes:


,id,labels,name,color,level,module,activity
0,Account Payable_1_5,[Task],Account Payable,#FFFACD,1,Account Payable,None
1,Account Payable_1_3,[StartEvent],StartEvent,#98FB98,1,Account Payable,None
2,Account Payable_1_7,[EndEvent],EndEvent,#FF4500,1,Account Payable,None
3,Account Payable_2_24,[Task],Reviewing Purchase Return,#FAFAD2,2,Account Payable,None
4,Account Payable_2_25,[Task],Approving Purchase Return,#FAFAD2,2,Account Payable,None



Relationships:


,id,type,source,target,color,level,module,activity
0,Account Payable_1_4,SEQUENCE_FLOW,Account Payable_1_5,Account Payable_1_7,#A9A9A9,1.0,Account Payable,None
1,Account Payable_1_6,SEQUENCE_FLOW,Account Payable_1_3,Account Payable_1_5,#A9A9A9,1.0,Account Payable,None
2,Account Payable_2_23,SEQUENCE_FLOW,Account Payable_2_24,Account Payable_2_25,#A9A9A9,2.0,Account Payable,None
3,Account Payable_2_34,SEQUENCE_FLOW,Account Payable_2_25,Account Payable_2_29,#A9A9A9,2.0,Account Payable,None
4,Account Payable_2_27,SEQUENCE_FLOW,Account Payable_2_28,Account Payable_2_29,#A9A9A9,2.0,Account Payable,None


neo4j_graph.html
